In [10]:
import numpy as np
import pandas as pd
import json
import nltk
from pymongo import MongoClient
import re

### Load Database

In [11]:
pd.set_option('display.max_colwidth', -1)

#loadDB
client = MongoClient()
db = client.uva_FODS
tweetsdb = db.tweets

### Query Database 
Filter only US english tweets and store to Pandas DataFrame

In [12]:
filter_query = {
    "$and":[ {"place.country_code":"US"}, { "lang": "en" } ]
    }
#we are keeping only our fields of interest
columns_query = {
    'text':1,
    'entities.hashtags':1,
    'entities.user_mentions':1,
    'place.full_name':1,
    'place.bounding_box':1
}

tweets = pd.DataFrame(list(tweetsdb.find(
    filter_query,
    columns_query
    )))#.limit(100000)

In [13]:
tweets['text'].head()

0    @BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3
1    #CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion 
2    @realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm 
3    "Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being" @funnyordie @realDonaldTrump😂💩s
4    @HillaryClinton you ARE the co-founder of ISIS, you crooked, evil, lying, witch. How can you live with yourself?                            
Name: text, dtype: object

### Text Preprocessing

-Tokenize
-Lemmatize
-Remove Punctuation

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer
import time

stop_words = list(nltk.corpus.stopwords.words('english'))
stop_words.extend(['trump','hillary','clinton', 
                   'american', 'republican', 'amp', 'usa']) #not needed for topic extraction

regexp_tokenizer = RegexpTokenizer(r'\w+')

In [14]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def preprocess_text(tweet):
    cleaned_tweet_text =""
    result = re.sub(r"\@\w+" , "", tweet) #remove hashtags
    result = re.sub(r"\#\w+" , "", result) #remove mentions
    result = re.sub(r"http\S+", "", result) #remove links
    result = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", result) #remove numbers
    # cleaned_tweet_text = punct_tokenizer.tokenize(result.lower())
    cleaned_tweet_text = regexp_tokenizer.tokenize(result.lower())
    cleaned_tweet_text = [token for token in cleaned_tweet_text if token not in stop_words]
    cleaned_tweet_text = [get_lemma(token) for token in cleaned_tweet_text]
    cleaned_tweet_text = [token for token in cleaned_tweet_text]
    finalized_tweet_text = []
    for word in cleaned_tweet_text:
        if (word is not '') and (len(word)>4 or word.lower() in stopwords.words('english')):
            finalized_tweet_text.append(word)
    return finalized_tweet_text

start_time = time.time()
tweets['cleanText'] = tweets['text'].apply(lambda x: preprocess_text(x)) #replace values 

print(elapsed_time)

232.29461073875427


In [15]:
tweets['cleanText'] = tweets['cleanText'].apply(lambda x: remove_stopwords(x))
tweets['cleanText'] .to_pickle('data/LDA/clean_data_for_LDA_' +str(len(tweets['text'])) +'.pkl')

517724